In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def convert_to_datetime(date_string):
    norsk_to_engelsk = {
        'mandag': 'Monday', 'tirsdag': 'Tuesday', 'onsdag': 'Wednesday',
        'torsdag': 'Thursday', 'fredag': 'Friday', 'lørdag': 'Saturday',
        'søndag': 'Sunday', 'januar': 'January', 'februar': 'February',
        'mars': 'March', 'april': 'April', 'mai': 'May', 'juni': 'June',
        'juli': 'July', 'august': 'August', 'september': 'September',
        'oktober': 'October', 'november': 'November', 'desember': 'December'
    }
    for norsk, engelsk in norsk_to_engelsk.items():
        date_string = date_string.replace(norsk, engelsk)
    return datetime.strptime(date_string + ' 2024', '%A %d. %B %Y')

def scrape_numbers():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    hovedtall = [int(li.text.strip()) for li in soup.select('ul li.h-10.w-10.rounded-full')][:5]
    stjernetall = [int(li.text.strip()) for li in soup.select('ul li.h-10.w-10.rounded-full')][5:]
    
    date = soup.select_one('span.font-medium.font-jubel').text.strip()
    
    return date, hovedtall, stjernetall

# Sett opp Chrome-opsjoner for headless-modus
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

# Sett opp Selenium WebDriver med Chrome-opsjoner
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.norsk-tipping.no/lotteri/eurojackpot/resultater")

results = []

while True:
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul li.h-10.w-10.rounded-full'))
        )
        
        date, hovedtall, stjernetall = scrape_numbers()
        results.append({'Dato': date, 'Hovedtall': hovedtall, 'Stjernetall': stjernetall})
        print(f"Scraped: {date}")
        
        previous_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "results-selected-draw-previous"))
        )
        previous_button.click()
        
    except TimeoutException:
        print("Kan ikke klikke mer eller siden lastet ikke. Avslutter.")
        break

driver.quit()

# Opprett DataFrame
results_df = pd.DataFrame(results)

# Konverter 'Dato' til datetime
results_df['Dato'] = results_df['Dato'].apply(convert_to_datetime)

# Sorter DataFrame etter dato i synkende rekkefølge
sorted_df = results_df.sort_values(by='Dato', ascending=True)

# Vis sortert DataFrame
print(sorted_df)

Scraped: tirsdag 17. september
Scraped: fredag 13. september
Scraped: tirsdag 10. september
Scraped: fredag 6. september
Scraped: tirsdag 3. september
Scraped: fredag 30. august
Scraped: tirsdag 27. august
Scraped: fredag 23. august
Scraped: tirsdag 20. august
Scraped: fredag 16. august
Scraped: tirsdag 13. august
Scraped: fredag 9. august
Scraped: tirsdag 6. august
Scraped: fredag 2. august
Scraped: tirsdag 30. juli
Scraped: fredag 26. juli
Scraped: tirsdag 23. juli
Scraped: fredag 19. juli
Scraped: tirsdag 16. juli
Scraped: fredag 12. juli
Scraped: tirsdag 9. juli
Scraped: fredag 5. juli
Scraped: tirsdag 2. juli
Scraped: fredag 28. juni
Scraped: tirsdag 25. juni
Scraped: fredag 21. juni
Scraped: tirsdag 18. juni
Scraped: fredag 14. juni
Scraped: tirsdag 11. juni
Scraped: fredag 7. juni
Kan ikke klikke mer eller siden lastet ikke. Avslutter.
         Dato             Hovedtall Stjernetall
29 2024-06-07   [8, 15, 29, 37, 45]     [5, 10]
28 2024-06-11   [4, 12, 16, 29, 31]      [1, 9]
2

In [19]:
import pandas as pd
from collections import Counter
import random

# Anta at sorted_df allerede er definert og inneholder dataene

# Flate ut hovedtall og stjernetall for analyse
hovedtall_flat = [num for sublist in sorted_df['Hovedtall'] for num in sublist]
stjernetall_flat = [num for sublist in sorted_df['Stjernetall'] for num in sublist]

# Tell forekomster av hvert tall
hovedtall_counts = Counter(hovedtall_flat)
stjernetall_counts = Counter(stjernetall_flat)

# Finn de minst vanlige (kalde) tallene
minst_vanlige_hovedtall = [num for num, count in hovedtall_counts.most_common()[-10:]]
minst_vanlige_stjernetall = [num for num, count in stjernetall_counts.most_common()[-5:]]

# Generer forslag til neste kombinasjon med et element av tilfeldighet
def generer_forslag(hovedtall, stjernetall):
    forslag_hovedtall = sorted(random.sample(hovedtall, 5))
    forslag_stjernetall = sorted(random.sample(stjernetall, 2))
    return forslag_hovedtall, forslag_stjernetall

forslag_1_hovedtall, forslag_1_stjernetall = generer_forslag(minst_vanlige_hovedtall, minst_vanlige_stjernetall)
forslag_2_hovedtall, forslag_2_stjernetall = generer_forslag(minst_vanlige_hovedtall, minst_vanlige_stjernetall)

print("Forslag 1 (Kalde tall):")
print(f"Hovedtall: {forslag_1_hovedtall}")
print(f"Stjernetall: {forslag_1_stjernetall}")

print("\nForslag 2 (Kalde tall):")
print(f"Hovedtall: {forslag_2_hovedtall}")
print(f"Stjernetall: {forslag_2_stjernetall}")

Forslag 1 (Kalde tall):
Hovedtall: [6, 12, 17, 20, 28]
Stjernetall: [4, 7]

Forslag 2 (Kalde tall):
Hovedtall: [6, 20, 28, 38, 41]
Stjernetall: [8, 9]
